In [12]:
import pandas as pd
import sqlite3


# *Dataset Loading*

In [13]:
orders = pd.read_csv("/content/orders - orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [14]:
users = pd.read_json("/content/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [15]:
conn = sqlite3.connect(":memory:")

with open("/content/restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


# *Data Inspection*

In [16]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [17]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


# *Questions & Pandas Queries*

# **Multiple Choice Questions**

1.Which city has the highest total revenue from Gold members?

In [18]:
df[df['membership']=='Gold'].groupby('city')['total_amount'].sum()


,total_amount
city,
Bangalore,994702.59
Chennai,1080909.79
Hyderabad,896740.19
Pune,1003012.32


2.Which cuisine has the highest average order value across all orders?

In [19]:
df.groupby('cuisine')['total_amount'].mean()


,total_amount
cuisine,
Chinese,798.389020
Indian,798.466011
Italian,799.448578
Mexican,808.021344


3.How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [43]:
count = df.groupby('user_id')['total_amount'].sum().gt(1000).sum()
count


np.int64(2544)

4.Which restaurant rating range generated the highest total revenue?

In [21]:
df['rating_range'] = pd.cut(
    df['rating'],
    bins=[3.0,3.5,4.0,4.5,5.0],
    labels=["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"]
)

df.groupby('rating_range')['total_amount'].sum() \
.sort_values(ascending=False)


/tmp/ipython-input-1094646158.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('rating_range')['total_amount'].sum() \


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


5.Among Gold members, which city has the highest average order value?

In [22]:
df[df['membership']=="Gold"] \
.groupby('city')['total_amount'].mean() \
.sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


6.Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [23]:
df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id','nunique'),
    revenue=('total_amount','sum')
).sort_values(['restaurant_count','revenue'], ascending=[True,False])


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


7.What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [24]:
percentage = round(
    (df[df['membership']=="Gold"].shape[0] / df.shape[0]) * 100
)
percentage


50

8.Which restaurant has the highest average order value but less than 20 total orders?

In [34]:
df.groupby('restaurant_name').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).query("total_orders < 20") \
 .sort_values('avg_order_value', ascending=False)


,total_orders,avg_order_value
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


9.Which combination contributes the highest revenue?

In [28]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

10.During which quarter of the year is the total revenue highest?

In [25]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'].sum() \
.sort_values(ascending=False)


/tmp/ipython-input-3312447427.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


# Numerical Answers

1.How many total orders were placed by users with Gold membership?


In [37]:
df[df["membership"] == "Gold"]["order_id"].nunique()


4987

2.What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [38]:
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

3.How many distinct users placed at least one order?

In [39]:
df["user_id"].nunique()


2883

4.What is the average order value (rounded to 2 decimals) for Gold members?

In [40]:
round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

5.How many orders were placed for restaurants with rating ≥ 4.5?

In [41]:
df[df["rating"] >= 4.5]["order_id"].nunique()


3374

6.How many orders were placed in the top revenue city among Gold members only?

In [42]:
top_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

df[(df["membership"] == "Gold") & (df["city"] == top_city)]["order_id"].nunique()


1337

# Final Answers

**Final Answers-MCQ's**

1.Which city has the highest total revenue (total_amount) from Gold members?

Ans-Chennai

2.Which cuisine has the highest average order value across all orders?

Ans-Mexican

3.How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
Ans-grater than 2000

4.Which restaurant rating range generated the highest total revenue?

Ans-4.6 – 5.0

5.Among Gold members, which city has the highest average order value?

Ans-Chennai

6.Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

Ans-Chinese

7.What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
Ans-50%

8.Which restaurant has the highest average order value but less than 20 total orders?

Ans-Hotel Dhaba Multicuisine

9.Which combination contributes the highest revenue?

Ans-Regular + Mexican cuisine

10.During which quarter of the year is the total revenue highest?

Ans-Q3 (Jul–Sep)

**Final Answers-Numerical Answers**

Total Gold Orders: 4987

Hyderabad Revenue: 1889367

Distinct Users: 2883

Gold AOV: 797.15

High Rating Orders: 3374

Top Gold City Orders: 1337

# **Fill in the blanks**

1.The column used to join orders.csv and users.json is
**user_id**

2.The dataset containing cuisine and rating information is stored in
**CSV format**

3.The total number of rows in the final merged dataset is
**1000**

4.If a user has no matching record in users.json, the merged values will be
**NaN (null values)**

5.The Pandas function used to combine datasets based on a key is
**merge()**

6.The column membership in the final dataset originates from the
**users.json** file

7.The join key used to combine orders data with restaurant details is
**restaurant_id**

8.The column that helps identify the type of food served by a restaurant is
**cuisine**

9.If a user places multiple orders, their personal details appear
**multiple** times in the final merged dataset